In [1]:
import tensorflow as tf

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def lrelu(x, factor = 0.2):
    return tf.maximum(x * 0.2, x)

In [3]:
def linear(input_, output_size, scope=None, stddev=0.02, bias_start=0.0):
    shape = input_.get_shape().as_list()
    with tf.variable_scope(scope or "Linear"):
        matrix = tf.get_variable("Matrix", [shape[1], output_size], tf.float32, 
            tf.random_normal_initializer(stddev=stddev))
        bias = tf.get_variable("bias", [output_size],
            initializer=tf.constant_initializer(bias_start))
        return tf.matmul(input_, matrix) + bias

In [4]:
l = tf.Variable(tf.random_normal([1, 100]))

In [5]:
x = linear(l, 1024, 'g_h0_lin')

In [6]:
x = tf.nn.relu(x)

In [7]:
x = linear(x, 512, 'g_h1_lin')

In [8]:
x = tf.nn.relu(x)

In [9]:
x = tf.reshape(x, [1, 2, 1, 256])

In [10]:
def deconv2d(input_, output_shape,
             k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02,
             name="deconv2d", with_w=False, pad = 'VALID'):
    with tf.variable_scope(name):
        # filter : [height, width, output_channels, in_channels]
        w = tf.get_variable('w', [k_h, k_w, output_shape[-1], input_.get_shape()[-1]],
                            initializer=tf.random_normal_initializer(stddev=stddev))
        
        try:
            deconv = tf.nn.conv2d_transpose(input_, w, output_shape=output_shape,
                                strides=[1, d_h, d_w, 1],padding = pad)

        # Support for verisons of TensorFlow before 0.7.0
        except AttributeError:
            deconv = tf.nn.deconv2d(input_, w, output_shape=output_shape,
                                strides=[1, d_h, d_w, 1],padding = pad)

        biases = tf.get_variable('biases', [output_shape[-1]], initializer=tf.constant_initializer(0.0))
        deconv = tf.reshape(tf.nn.bias_add(deconv, biases), deconv.get_shape())

        if with_w:
            return deconv, w, biases
        else:
            return deconv


In [11]:
x = deconv2d(x, [1, 4, 1, 256],k_h=2, k_w=1,d_h=2, d_w=2 ,name='g_h2')

In [12]:
x.get_shape()

TensorShape([Dimension(1), Dimension(4), Dimension(1), Dimension(256)])

In [13]:
a = tf.Variable(tf.random_normal([1,100]))

In [26]:
b = tf.Variable(tf.random_normal([1,20]))

In [27]:
sess = tf.InteractiveSession()

In [28]:
init = tf.global_variables_initializer()
sess.run(init)

In [30]:
c = tf.concat([a,b],1)

In [31]:
c.get_shape()

TensorShape([Dimension(1), Dimension(120)])

In [34]:
c.get_shape()

TensorShape([Dimension(1), Dimension(120)])

In [101]:
def conv_cond_concat(x, y):
    """Concatenate conditioning vector on feature map axis."""
    x_shapes = x.get_shape().as_list()
    y_shapes = y.get_shape().as_list()
    return tf.concat([x, y*tf.ones([x_shapes[0], x_shapes[1], x_shapes[2], y_shapes[3]])], 3)

In [102]:
def conv_prev_concat(x, y):
    """Concatenate conditioning vector on feature map axis."""
    x_shapes = x.get_shape().as_list()
    y_shapes = y.get_shape().as_list()
    if x_shapes[:2] == y_shapes[:2]:
        return tf.concat([x, y*tf.ones([x_shapes[0], x_shapes[1], x_shapes[2], y_shapes[3]])], 3)
    else:
        print(x_shapes[:2])
        print(y_shapes[:2])

In [90]:
test2 = tf.random_normal([1, 1, 1, 128])

In [91]:
test1 = tf.random_normal([1, 1, 128, 20])

In [105]:
shapes1 = test1.get_shape().as_list()
shapes2 = test2.get_shape().as_list()

In [190]:
test3 = (test2 * tf.ones([shapes1[0], shapes1[1], shapes1[2], shapes2[3]]))

In [107]:
test2.get_shape()

TensorShape([Dimension(1), Dimension(1), Dimension(1), Dimension(128)])

In [179]:
test3 = conv_cond_concat(test1, test2)

In [195]:
sess = tf.InteractiveSession()
tf.ones([shapes1[0], shapes1[1], shapes1[2], shapes2[3]]).get_shape()

TensorShape([Dimension(1), Dimension(1), Dimension(128), Dimension(128)])

In [191]:
test4 = sess.run(test3)

In [194]:
test4[0][0][1] - test4[0][0][2]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.], dtype=float32)

In [83]:
test2.get_shape().as_list()[:2] == x_shapes[:2]

True

In [138]:
def conv2d(input_, output_dim, 
           k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02,
           name="conv2d"):
    with tf.variable_scope(name, reuse = tf.AUTO_REUSE):
        w = tf.get_variable('w', [k_h, k_w, input_.get_shape()[-1], output_dim],
                            initializer=tf.truncated_normal_initializer(stddev=stddev))
        conv = tf.nn.conv2d(input_, w, strides=[1, d_h, d_w, 1], padding='VALID')

        biases = tf.get_variable('biases', [output_dim], initializer=tf.constant_initializer(0.0))
        conv = tf.reshape(tf.nn.bias_add(conv, biases), conv.get_shape())

        return conv

In [155]:
testinput = tf.random_normal([1,16,128,1])

In [129]:
testinput2 = conv2d(testinput, 16, k_h=1, k_w=128,d_h=1, d_w=2, name='2')

In [135]:
testinput3 = conv2d(testinput2, 16, k_h = 2, k_w = 1, name = '3.')

In [136]:
print(testinput2.get_shape())
print(testinput3.get_shape())

(1, 1, 64, 16)
(1, 1, 32, 16)


In [141]:
conv2d(testinput2, 16, k_h = 1, k_w = 2, name = 'test4')

<tf.Tensor 'test4/Reshape:0' shape=(1, 1, 32, 16) dtype=float32>

In [147]:
tf.random_normal([5, 5, testinput.get_shape()[-1].value, 5])

<tf.Tensor 'random_normal_21:0' shape=(5, 5, 16, 5) dtype=float32>

In [196]:
filter1 = tf.Variable(tf.random_normal(shape = [1,128,1,16], stddev = 0.02))
bias1 = tf.Variable(tf.constant(0.0, tf.float32, shape = [16]))
conv1 = tf.nn.conv2d(testinput, filter1, strides = [1,1,2,1], padding = 'VALID')

In [197]:
conv1.get_shape()

TensorShape([Dimension(1), Dimension(16), Dimension(1), Dimension(16)])

In [176]:
filter2 = tf.Variable(tf.random_normal(shape = [2,1,16,16], stddev = 0.02))
bias2 = tf.Variable(tf.constant(0.0, tf.float32, shape = [16]))
conv2 = tf.nn.conv2d(conv1, filter2, strides = [1,2,2,1], padding = 'VALID')

In [177]:
conv2.get_shape()

TensorShape([Dimension(1), Dimension(4), Dimension(1), Dimension(16)])

In [178]:
tf.nn.bias_add(conv2, bias2).get_shape()

TensorShape([Dimension(1), Dimension(4), Dimension(1), Dimension(16)])